In [ ]:
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 6.3MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144800 sha256=384e31ee61a3b6fada48ed7f2af053fe6d6002d5d760e6047937675b15af48a9
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [ ]:
import librosa
import numpy as np
import pdb
import string
from Levenshtein import distance

import os
from collections import Counter
import tensorflow.keras as keras
import tensorflow as tf
import sklearn
import random
import itertools
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:

def wav2feat(wavfile):
    '''
    Input: audio wav file name
    Output: Magnitude spectrogram
    '''
    x, Fs = librosa.load(wavfile, sr=44100, mono=True) 
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X)

def wavs2feat(wavfiles):
    '''
    Concatenate the audio files listed in wavfiles
    Input: list of audio wav file names
    Output: Magnitude spectrogram of concatenated wav
    '''
    x = []
    for wf in wavfiles:
        x1, Fs = librosa.load(wf, sr=44100, mono=True)
        x.append(x1)
    x = np.hstack(x)
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X)

def read_csv(filename):
    id_label = {}
    with open(filename,'r') as fid:
        for line in fid: # '176787-5-0-27.wav,engine_idling\n'
            tokens = line.strip().split(',') # ['176787-5-0-27.wav', 'engine_idling']
            id_label[tokens[0]] = tokens[1]
    return id_label

def editDistance(gt, est):
    '''both are lists of labels
    E.g. gt is "dog_bark-street_music-engine_idling"
    E.g. est is "street_music-engine_idling"
    '''
    gttokens = gt.split('-')
    esttokens = est.split('-')
    # Map token to char
    tokenset = list(set(gttokens+esttokens)) # ['dog_bark', 'siren', 'street_music', 'engine_idling']
    token_char = {}
    for i in range(len(tokenset)):
        token_char[tokenset[i]] = string.ascii_uppercase[i]  # {'dog_bark': 'A', 'siren': 'B', 'street_music': 'C', 'engine_idling': 'D'}
    # convert gt and est to strings
    gtstr = [token_char[t] for t in gttokens]
    gtstr = ''.join(gtstr)  # 'BCA'
    eststr = [token_char[t] for t in esttokens]
    eststr = ''.join(eststr)  # 
    # Compare
    editdist = distance(gtstr, eststr) # 1
    score = 1 - editdist/len(gtstr)
    return editdist, score

def evals(gtcsv, estcsv, taskid):
    gt_id_label = read_csv(gtcsv)
    est_id_label = read_csv(estcsv)
    score = 0
    for id in est_id_label:
        if taskid==1:
            if est_id_label[id] == gt_id_label[id]:
                score += 1
        elif taskid==2:
            _, ss = editDistance(gt_id_label[id], est_id_label[id])
            score += ss
        else:
            pdb.set_trace()
            assert False, ["taskid not correct; it is", taskid]
    avgScore = score/len(est_id_label)
    return avgScore


In [ ]:
name_label_dict = read_csv('/content/drive/My Drive/AED/labels_train.csv')
labels_list = list(set(name_label_dict.values()))
train_set_dist = Counter(name_label_dict.values())
label_to_idx = {'jackhammer':0, 'drilling':1, 'dog_bark':2, 'air_conditioner':3, 'engine_idling':4, 'children_playing':5, 'siren':6, 'gun_shot':7, 'car_horn':8, 'street_music':9}
idx_to_label = {0:'jackhammer', 1:'drilling', 2:'dog_bark', 3:'air_conditioner', 4:'engine_idling', 5:'children_playing', 6:'siren', 7:'gun_shot', 8:'car_horn', 9:'street_music'}
print(labels_list, train_set_dist)

['car_horn', 'drilling', 'siren', 'air_conditioner', 'class', 'gun_shot', 'dog_bark', 'street_music', 'engine_idling', 'jackhammer', 'children_playing'] Counter({'jackhammer': 240, 'dog_bark': 200, 'drilling': 200, 'children_playing': 200, 'street_music': 200, 'air_conditioner': 200, 'engine_idling': 196, 'siren': 177, 'car_horn': 78, 'gun_shot': 70, 'class': 1})


In [ ]:
path = '/content/drive/My Drive/audio_train_1ch'
files=[]
files = [f for f in sorted(os.listdir(path))]
X_train_dict = {'jackhammer':[], 'drilling':[], 'dog_bark':[], 'air_conditioner':[], 'class':[], 'children_playing':[], 'siren':[], 'gun_shot':[], 'car_horn':[], 'street_music':[], 'engine_idling':[]}

for f in files:
  wavfile = [path + '/' + f]
  x = wavs2feat(wavfile)
  X_train_dict[name_label_dict[f]].append(x)
  if x.shape[1] != 401 or x.shape[0] != 513:
    print(f, x.shape, name_label_dict[f])

In [ ]:
with open('/content/drive/My Drive/AED/X_train_dict', 'rb') as handle:
    X_train_dict = pickle.load(handle)

In [ ]:
X_train = []
targets = []
for a,b in X_train_dict.items():
  for c in b:
    X_train.append(c)
    targets.append(label_to_idx[a])

nb_classes = 10
targets = np.array(targets).reshape(-1)
Y_train = np.eye(nb_classes)[targets]

X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X_train, Y_train, test_size=0.10, random_state=42)

print(len(X_train))

1584


In [ ]:
def accuracy(true, pred):
  n = true.shape[0]
  correct = 0
  for i in range(n):
    if (np.argmax(true[i]) == np.argmax(pred[i]))  and pred[i][np.argmax(pred[i])] >= 0.5:
      correct = correct+1
  return correct/n

In [ ]:
def model():
  inp = keras.layers.Input(shape = (None, 513))
  out = tf.keras.layers.LSTM(100, return_sequences=True)(inp)
  out = tf.keras.layers.LSTM(100)(out)
  out = tf.keras.layers.Dense(10, activation = 'softmax')(out)
  mod = tf.keras.Model(inp, out)
  return mod

In [ ]:
pred = model()
pred.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 513)]       0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 100)         245600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 10)                1010      
Total params: 327,010
Trainable params: 327,010
Non-trainable params: 0
_________________________________________________________________


In [ ]:
pred = tf.keras.models.load_model('/content/drive/My Drive/AED/task1_final_mod.hdf5')

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate = 0.001)
cce = tf.keras.losses.CategoricalCrossentropy()
no_of_samples = len(X_train)
for i in range(1000):
  print('iteration ;:::: ', i)
  loss = 0
  predicted = []
  no_of_samples = len(X_train)
  with tf.GradientTape() as Tape:
    for k in range(no_of_samples):
      inp = X_train[k]
      inp = np.expand_dims(np.transpose(inp), axis = 0)
      out = np.expand_dims(Y_train[k], axis = 0)
      output_ = pred(inp)
      predicted.append(output_.numpy())
      loss_temp =  tf.reduce_mean(cce(out, output_))
      loss += loss_temp
      #print(out, output_, loss_temp)
    loss = loss/no_of_samples
  predicted = np.concatenate(predicted, axis = 0)
  print('TRAIN :;;;;;: ', accuracy(Y_train[0:no_of_samples,:], predicted), loss)
  grad = Tape.gradient(loss, pred.trainable_variables)
  optimizer.apply_gradients(zip(grad, pred.trainable_variables))

  ## validation
  loss = 0
  predicted = []
  no_of_samples = len(X_test)
  for k in range(no_of_samples):
    inp = X_test[k]
    inp = np.expand_dims(np.transpose(inp), axis = 0)
    out = np.expand_dims(Y_test[k], axis = 0)
    output_ = pred(inp)
    predicted.append(output_.numpy())
    loss_temp =  tf.reduce_mean(cce(out, output_))
    loss += loss_temp
    #print(out, output_, loss_temp)
  loss = loss/no_of_samples
  predicted = np.concatenate(predicted, axis = 0)
  print('TEST :;;;: ', accuracy(Y_test[0:no_of_samples,:], predicted), loss)


  # SAVE
  pred.save('/content/drive/My Drive/AED/task1_final_mod.hdf5')

iteration ;::::  0
TRAIN :;;;;;:  0.5435606060606061 tf.Tensor(1.0047196, shape=(), dtype=float32)
TEST :;;;:  0.5932203389830508 tf.Tensor(0.9817929, shape=(), dtype=float32)
iteration ;::::  1
TRAIN :;;;;;:  0.6527777777777778 tf.Tensor(0.8354409, shape=(), dtype=float32)
TEST :;;;:  0.6949152542372882 tf.Tensor(0.8255299, shape=(), dtype=float32)
iteration ;::::  2
TRAIN :;;;;;:  0.6856060606060606 tf.Tensor(0.75145906, shape=(), dtype=float32)
TEST :;;;:  0.7344632768361582 tf.Tensor(0.73409325, shape=(), dtype=float32)
iteration ;::::  3
TRAIN :;;;;;:  0.7329545454545454 tf.Tensor(0.65884274, shape=(), dtype=float32)
TEST :;;;:  0.7627118644067796 tf.Tensor(0.6913882, shape=(), dtype=float32)
iteration ;::::  4
TRAIN :;;;;;:  0.7348484848484849 tf.Tensor(0.63609517, shape=(), dtype=float32)
TEST :;;;:  0.7909604519774012 tf.Tensor(0.68740666, shape=(), dtype=float32)
iteration ;::::  5
TRAIN :;;;;;:  0.7702020202020202 tf.Tensor(0.57443136, shape=(), dtype=float32)
TEST :;;;:  0.7

KeyboardInterrupt: ignored